In [7]:
from graphviz import Digraph

# Use the Neato engine so we can specify exact (x,y) positions.
dot = Digraph('PFSFlowchart', engine='neato', format='png')

# Global graph settings for appearance and spacing
dot.attr(overlap='True', splines='ortho')
dot.attr('graph', nodesep='0.3', ranksep='0.3')

# Define styles for process boxes (light-blue) and data cylinders (light-blue)
PROCESS_STYLE = {
    'shape': 'box',
    'style': 'filled',
    'fillcolor': 'white',
    'color': 'black',
    'fontname': 'Helvetica',
    'width': '2', 
    'height': '0.6',
    'fixedsize': 'true'
}
DATA_STYLE = {
    'shape': 'cylinder',
    'style': 'filled',
    'fillcolor': '#B7E1F7',
    'color': 'black',
    'fontname': 'Helvetica',
    'width': '1.5', 
    'height': '0.6',
    'fixedsize': 'true'
}

# 1) Top-left input node: raw, bias, dark, flat, ...
dot.node('raw', label='raw, bias,\ndark, flat, ...', pos='0,0!', **DATA_STYLE)

# 2) reduceExposure process
dot.node('reduceExposure', label='reduceExposure', pos='3.3,0!', **PROCESS_STYLE)

# 3) pfsArm (top)
dot.node('pfsArmTop', label='pfsArm', pos='6,0!', **DATA_STYLE)

# 3.5) pfsArms
dot.node('pfsArms_back2', label='', pos='0.5, -1.3!', **DATA_STYLE)
dot.node('pfsArms_back1', label='', pos='0.6, -1.4!', **DATA_STYLE)
dot.node('pfsArms',       label='pfsArms', pos='0.7,-1.5!',   **DATA_STYLE)

# 4) mergeArms process (below pfsArm)
dot.node('mergeArms', label='mergeArms', pos='3.3,-1.5!', **PROCESS_STYLE)

# 5) sky1d (to the right of mergeArms)
dot.node('sky1dTop', label='sky1d', pos='6,-1.1!', **DATA_STYLE)

# 6) pfsMerged (below sky1d)
dot.node('pfsMerged', label='pfsMerged', pos='6,-1.9!', **DATA_STYLE)

# 7) fitPfsFluxReference process
dot.node('fitPfsFluxReference', label='fitPfsFluxReference', pos='1,-3!', **PROCESS_STYLE)

# 8) pfsReference (below fitPfsFluxReference)
dot.node('pfsReference', label='pfsReference', pos='6,-3!', **DATA_STYLE)

# 9) fitFluxCal process
dot.node('fitFluxCal', label='fitFluxCal', pos='1.2,-4.5!', **PROCESS_STYLE)

# 10) pfsCalibrated (top-right from fitFluxCal)
dot.node('pfsCalibrated', label='pfsCalibrated', pos='6,-4.1!', **DATA_STYLE)

# 11) fluxCal (below pfsSingle)
dot.node('fluxCalTop', label='fluxCal', pos='6,-4.9!', **DATA_STYLE)

# 12) Repeated nodes for bottom row (to show multi-frame concept):
#     pfsArm, sky1d, fluxCal feed into coaddSpectra
dot.node('sky1dBot_back2', label='', pos='0.5, -5.9!', **DATA_STYLE)
dot.node('sky1dBot_back1', label='', pos='0.6, -6.0!', **DATA_STYLE)
dot.node('sky1dBot',       label='sky1d', pos='0.7,-6.1!',   **DATA_STYLE)

dot.node('fluxCalBot_back2', label='', pos='0.5, -6.8!', **DATA_STYLE)
dot.node('fluxCalBot_back1', label='', pos='0.6, -6.9!', **DATA_STYLE)
dot.node('fluxCalBot',       label='fluxCal', pos='0.7,-7.0!',   **DATA_STYLE)

dot.node('pfsArmBot_back2', label='', pos='0.5, -7.7!', **DATA_STYLE)
dot.node('pfsArmBot_back1', label='', pos='0.6, -7.8!', **DATA_STYLE)
dot.node('pfsArmBot',       label='pfsArm', pos='0.7,-7.9!',   **DATA_STYLE)

# 13) coaddSpectra process
dot.node('coaddSpectra', label='coaddSpectra', pos='3.3,-7.0!', **PROCESS_STYLE)

# 14) pfsObject (final product)
dot.node('pfsCoadd', label='pfsCoadd', pos='6,-7.0!', **DATA_STYLE)

#---------------------
# EDGES (data flow)
#---------------------

# Top row flow: raw -> reduceExposure -> pfsArm
dot.edge('raw', 'reduceExposure')
dot.edge('reduceExposure', 'pfsArmTop')

# pfsArm -> mergeArms -> (sky1d, pfsMerged)
dot.edge('pfsArmTop', 'pfsArms_back2')
dot.edge('pfsArms', 'mergeArms')
dot.edge('mergeArms', 'sky1dTop')
dot.edge('mergeArms', 'pfsMerged')

# pfsMerged -> fitPfsFluxReference -> pfsReference
dot.edge('pfsMerged', 'fitPfsFluxReference')
dot.edge('fitPfsFluxReference', 'pfsReference')

# pfsReference -> fitFluxCal -> (pfsCalibrated, fluxCal)
dot.edge('pfsReference', 'fitFluxCal')
dot.edge('fitFluxCal', 'pfsCalibrated')
dot.edge('fitFluxCal', 'fluxCalTop')

# Show that bottom pfsArm/sky1d/fluxCal are conceptually the same as top
# (use invisible edges to tie them together)
dot.edge('pfsArmTop', 'pfsArmBot_back2')
dot.edge('sky1dTop', 'sky1dBot_back2', style='invis')
dot.edge('fluxCalTop', 'fluxCalBot_back2', style='invis')

# coaddSpectra receives multiple frames: pfsArmBot, sky1dBot, fluxCalBot
dot.edge('pfsArmBot', 'coaddSpectra')
dot.edge('sky1dBot', 'coaddSpectra')
dot.edge('fluxCalBot', 'coaddSpectra')

# Final output: pfsCoadd
dot.edge('coaddSpectra', 'pfsCoadd')

# Render and open
# Render and open
dot.render('../img/pipe2d_flowchart_gen3', view=True)


'../img/pipe2d_flowchart_gen3.png'